In [2]:
!pip install gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 MB 52.2 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.2/11.2 MB 65.1 MB/s eta 0:00:00
  Attempting uninstall: markupsafe
    Found existing installation: MarkupSafe 3.0.2
    Uninstalling MarkupSafe-3.0.2:
      Successfully uninstalled MarkupSafe-3.0.2


In [12]:
import faiss
import numpy as np
import gradio as gr
import pickle
from openai import OpenAI
from credentials.keys import OPENAI_API_KEY

print(faiss.__version__)
print("FAISS sur cpu ou gpu ?"+str(faiss.get_num_gpus()))  # Si 0 → FAISS utilise uniquement le CPU

# Chargement de l'index FAISS et des métadonnées
index = faiss.read_index("faiss_index_openai.bin")
with open("metadata_openai.pkl", "rb") as f:
    metadata = pickle.load(f)

# Initialisation du client OpenAI
client = OpenAI(api_key=OPENAI_API_KEY)


# Fonction pour générer l'URL externe d'une série
def generate_series_url(series_key, dataset_id):
    base_url = "https://webstat.banque-france.fr/fr/catalogue/"
    return f"{base_url}{dataset_id}/{series_key}"

    
# Fonction de recherche et calcul de distance spécifique
def search_query(query_text, series_key):
    if not query_text:
        return "Veuillez entrer une requête."
    
    # Génération de l'embedding de la requête
    response = client.embeddings.create(
        input=query_text,
        model="text-embedding-3-small"
    )
    query_embedding = np.array(response.data[0].embedding, dtype=np.float32).reshape(1, -1)
    
   # Recherche dans FAISS
    k = 10  # Nombre de résultats à récupérer
    distances, indices = index.search(query_embedding, k)
    
    output = f"Phrase recherchée : {query_text}\n\n"
    
    for i, (distance, idx) in enumerate(zip(distances[0], indices[0])):
        if idx < len(metadata):  # Vérifier si l'index est valide
            doc_id = list(metadata.keys())[idx]
            meta_info = metadata[doc_id]
            dataset_id = meta_info.get('dataset_id', 'inconnu')
            series_url = generate_series_url(doc_id, dataset_id)
            output += (f"<br>Résultat {i+1}: <a href='{series_url}' target='_blank'>{doc_id}</a> (distance: {distance})<br>"
                       f"  - Dataset: {dataset_id}<br>"
                       f"  - Description dataset: {meta_info.get('description', '')}<br>"
                       f"  - Nom série: {meta_info.get('title', '')}<br>"
                       f"  - Périodicité série: {meta_info.get('periodicity', '')}<br><br>")
                       
    
    # Calcul de la distance avec une série spécifique si fournie
    specific_distance_info = ""
    if series_key and series_key in metadata:
        specific_embedding = np.array(response.data[0].embedding, dtype=np.float32).reshape(1, -1)
        specific_distance = np.linalg.norm(query_embedding - specific_embedding)
        dataset_id = metadata[series_key].get('dataset_id', 'inconnu')
        specific_url = generate_series_url(series_key, dataset_id)
        specific_distance_info = f"\n\nDistance entre la requête et '<a href='{specific_url}' target='_blank'>{series_key}</a>': {specific_distance}"
    else:
        specific_distance_info = f"\n\nL'ID '{series_key}' n'existe pas dans l'index FAISS."
    
    return output + specific_distance_info

# Interface Gradio
demo = gr.Interface(
    fn=search_query,
    inputs=[
        gr.Textbox(label="Entrez votre requête"),
        gr.Textbox(label="Entrez une series_key spécifique (optionnel)")
    ],
    outputs=gr.HTML(label="Résultats de la recherche"),
    title="Recherche de texte avec FAISS et OpenAI"
)

demo.launch(share=True)


"""
IMPRESSIONNANT
j'ai un projet immobilier, quels sont les taux actuels ? MIR1.M.FR.B.A2C.O.R.A.2250U6.EUR.N
en finnois
Minulla on kiinteistöprojekti, mitkä ovat nykyiset korot?

quelle est la conjoncture dans le vêtement en France ? CONJ.M.N01.S.IN.000CB.CCTSM000.10 ou CONJ.M.N01.S.IN.000CB.PVTSM000.10
taux de change pays anglophones   ; EXR.D.USD.EUR.SP00.A ou EXR.M.USD.EUR.SP00.A
taux de change pays de l'est ; XR.D.EEK.EUR.SP00.A ou EXR.D.RUB.EUR.SP00.A
évolution mensuelle des faillites d'entreprises ; DIREN.M.FR.DE.UL.DF.25.N.ZZ.TT



quel est le taux d'intérêt maximal légal qu'un établissement de crédit peut appliquer lorsqu'il accorde un prêt immobilier ? MIR1.M.FR.R.A22FRF.W3.U.A.2254FR.EUR.N

LOGIQUEMENT CORRECT
quel est l'encours total de la monaie fiduciaire ? BSI.M.U2.N.U.L10.X.1.U2.2300.Z01.E
Comment est l'inflation en france ? ICP.M.FR.N.000000.4.ANR

MOYENS
combien de personnes sont surendettées ?

MAUVAIS
taux de changes devises quotidiennes
on pourrait ajouter des valeurs des dimensions
"""



1.10.0
FAISS sur cpu ou gpu ?0
* Running on local URL:  http://127.0.0.1:7878
* Running on public URL: https://1914190c36e1cc41a5.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


"\nIMPRESSIONNANT\nj'ai un projet immobilier, quels sont les taux actuels ? MIR1.M.FR.B.A2C.O.R.A.2250U6.EUR.N\nen finnois\nMinulla on kiinteistöprojekti, mitkä ovat nykyiset korot?\n\nquelle est la conjoncture dans le vêtement en France ? CONJ.M.N01.S.IN.000CB.CCTSM000.10 ou CONJ.M.N01.S.IN.000CB.PVTSM000.10\ntaux de change pays anglophones   ; EXR.D.USD.EUR.SP00.A ou EXR.M.USD.EUR.SP00.A\ntaux de change pays de l'est ; XR.D.EEK.EUR.SP00.A ou EXR.D.RUB.EUR.SP00.A\névolution mensuelle des faillites d'entreprises ; DIREN.M.FR.DE.UL.DF.25.N.ZZ.TT\n\n\n\nquel est le taux d'intérêt maximal légal qu'un établissement de crédit peut appliquer lorsqu'il accorde un prêt immobilier ? MIR1.M.FR.R.A22FRF.W3.U.A.2254FR.EUR.N\n\nLOGIQUEMENT CORRECT\nquel est l'encours total de la monaie fiduciaire ? BSI.M.U2.N.U.L10.X.1.U2.2300.Z01.E\nComment est l'inflation en france ? ICP.M.FR.N.000000.4.ANR\n\nMOYENS\ncombien de personnes sont surendettées ?\n\nMAUVAIS\ntaux de changes devises quotidiennes\non po

Traceback (most recent call last):
  File "/opt/conda/lib/python3.12/site-packages/gradio/queueing.py", line 625, in process_events
    response = await route_utils.call_process_api(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/conda/lib/python3.12/site-packages/gradio/route_utils.py", line 322, in call_process_api
    output = await app.get_blocks().process_api(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/conda/lib/python3.12/site-packages/gradio/blocks.py", line 2108, in process_api
    result = await self.call_function(
             ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/conda/lib/python3.12/site-packages/gradio/blocks.py", line 1655, in call_function
    prediction = await anyio.to_thread.run_sync(  # type: ignore
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/conda/lib/python3.12/site-packages/anyio/to_thread.py", line 56, in run_sync
    return await get_async_backend().run_sync_in_worker_thread(
           ^^^^^